In [1]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

In [2]:
mt5.initialize()

True

In [34]:
def get_rates(symbol, timeframe, start_dt, end_dt):
    rates = mt5.copy_rates_range(symbol, timeframe, start_dt, end_dt)
    rates_df = pd.DataFrame(rates)

    rates_df['time'] = pd.to_datetime(rates_df['time'], unit='s')
    rates_df['perc'] = (rates_df['close'] - rates_df['open']) / rates_df['open']
    rates_df['perc_cumsum'] = rates_df['perc'].cumsum()

    return rates_df[['time', 'open', 'high', 'low', 'close', 'perc', 'perc_cumsum']]

In [44]:
start = datetime(2023, 1, 1)
end = datetime(2023, 10, 1)
timeframe = mt5.TIMEFRAME_D1

In [45]:
uj = get_rates('USDJPY', timeframe, start, end)
uj

,time,open,high,low,close,perc,perc_cumsum
0,2023-01-02,130.924,131.046,130.601,130.601,-0.002467,-0.002467
1,2023-01-03,130.698,131.400,129.517,130.985,0.002196,-0.000271
2,2023-01-04,131.014,132.716,129.926,132.652,0.012502,0.012231
3,2023-01-05,132.570,134.055,131.684,133.421,0.006419,0.018651
4,2023-01-06,133.406,134.774,131.998,132.106,-0.009745,0.008906
...,...,...,...,...,...,...,...
190,2023-09-25,148.252,148.966,148.201,148.893,0.004324,0.179077
191,2023-09-26,148.877,149.188,148.706,149.064,0.001256,0.180333
192,2023-09-27,149.053,149.710,148.862,149.647,0.003985,0.184319
193,2023-09-28,149.621,149.621,149.149,149.300,-0.002145,0.182173


In [57]:
dj = get_rates('US500', timeframe, start, end)
dj

,time,open,high,low,close,perc,perc_cumsum
0,2023-01-03,3866.3,3884.1,3793.1,3821.7,-0.011536,-0.011536
1,2023-01-04,3819.6,3873.6,3814.7,3848.0,0.007435,-0.004100
2,2023-01-05,3849.3,3862.6,3801.3,3811.4,-0.009846,-0.013946
3,2023-01-06,3813.5,3907.6,3797.9,3891.6,0.020480,0.006534
4,2023-01-09,3901.1,3951.0,3887.9,3891.3,-0.002512,0.004022
...,...,...,...,...,...,...,...
188,2023-09-25,4326.0,4342.6,4298.3,4342.4,0.003791,0.098313
189,2023-09-26,4340.3,4341.0,4265.5,4279.1,-0.014100,0.084212
190,2023-09-27,4281.2,4295.7,4238.1,4283.4,0.000514,0.084726
191,2023-09-28,4282.8,4318.2,4262.6,4303.8,0.004903,0.089629


In [58]:
jp = get_rates('JP225', timeframe, start, end)
jp

,time,open,high,low,close,perc,perc_cumsum
0,2023-01-03,25926.0,25975.0,25601.0,25828.0,-0.003780,-0.003780
1,2023-01-04,25803.0,25928.0,25618.0,25855.0,0.002015,-0.001765
2,2023-01-05,25853.0,25951.0,25701.0,25731.0,-0.004719,-0.006484
3,2023-01-06,25732.0,26219.0,25709.0,26189.0,0.017760,0.011276
4,2023-01-09,26189.0,26457.0,26162.0,26217.0,0.001069,0.012345
...,...,...,...,...,...,...,...
188,2023-09-25,32485.0,32743.0,32381.0,32708.0,0.006865,0.270674
189,2023-09-26,32692.0,32699.0,32048.0,32113.0,-0.017711,0.252963
190,2023-09-27,32136.0,32477.0,31952.0,32277.0,0.004388,0.257350
191,2023-09-28,32057.0,32165.0,31667.0,32009.0,-0.001497,0.255853


In [59]:
c_df = dj[['time', 'perc_cumsum']].merge(jp[['time', 'perc_cumsum']], on='time', suffixes=['_dj', '_jp'])
c_df['diff'] = c_df['perc_cumsum_dj'] - c_df['perc_cumsum_jp']
c_df

,time,perc_cumsum_dj,perc_cumsum_jp,diff
0,2023-01-03,-0.011536,-0.003780,-0.007756
1,2023-01-04,-0.004100,-0.001765,-0.002336
2,2023-01-05,-0.013946,-0.006484,-0.007462
3,2023-01-06,0.006534,0.011276,-0.004743
4,2023-01-09,0.004022,0.012345,-0.008324
...,...,...,...,...
188,2023-09-25,0.098313,0.270674,-0.172361
189,2023-09-26,0.084212,0.252963,-0.168751
190,2023-09-27,0.084726,0.257350,-0.172624
191,2023-09-28,0.089629,0.255853,-0.166224


In [60]:
px.line(c_df, x='time', y=['perc_cumsum_dj', 'perc_cumsum_jp'])

In [61]:
px.line(c_df, x='time', y='diff')

In [62]:
c_df2 = c_df.merge(uj[['time', 'perc_cumsum']], on='time')
c_df2 = c_df2.rename(columns={'perc_cumsum': 'perc_cumsum_uj'})

c_df2

,time,perc_cumsum_dj,perc_cumsum_jp,diff,perc_cumsum_uj
0,2023-01-03,-0.011536,-0.003780,-0.007756,-0.000271
1,2023-01-04,-0.004100,-0.001765,-0.002336,0.012231
2,2023-01-05,-0.013946,-0.006484,-0.007462,0.018651
3,2023-01-06,0.006534,0.011276,-0.004743,0.008906
4,2023-01-09,0.004022,0.012345,-0.008324,0.008603
...,...,...,...,...,...
188,2023-09-25,0.098313,0.270674,-0.172361,0.179077
189,2023-09-26,0.084212,0.252963,-0.168751,0.180333
190,2023-09-27,0.084726,0.257350,-0.172624,0.184319
191,2023-09-28,0.089629,0.255853,-0.166224,0.182173


In [63]:
px.line(c_df2, x='time', y=['perc_cumsum_dj', 'perc_cumsum_jp', 'perc_cumsum_uj'])

In [64]:
c_df2[['perc_cumsum_dj', 'perc_cumsum_jp', 'perc_cumsum_uj']].corr()

,perc_cumsum_dj,perc_cumsum_jp,perc_cumsum_uj
perc_cumsum_dj,1.000000,0.925461,0.811769
perc_cumsum_jp,0.925461,1.000000,0.917252
perc_cumsum_uj,0.811769,0.917252,1.000000


In [68]:
c_df2['tri_hedge'] = c_df2['perc_cumsum_uj'] + c_df2['perc_cumsum_dj']
c_df2

,time,perc_cumsum_dj,perc_cumsum_jp,diff,perc_cumsum_uj,tri_hedge
0,2023-01-03,-0.011536,-0.003780,-0.007756,-0.000271,-0.011807
1,2023-01-04,-0.004100,-0.001765,-0.002336,0.012231,0.008131
2,2023-01-05,-0.013946,-0.006484,-0.007462,0.018651,0.004704
3,2023-01-06,0.006534,0.011276,-0.004743,0.008906,0.015440
4,2023-01-09,0.004022,0.012345,-0.008324,0.008603,0.012624
...,...,...,...,...,...,...
188,2023-09-25,0.098313,0.270674,-0.172361,0.179077,0.277390
189,2023-09-26,0.084212,0.252963,-0.168751,0.180333,0.264546
190,2023-09-27,0.084726,0.257350,-0.172624,0.184319,0.269045
191,2023-09-28,0.089629,0.255853,-0.166224,0.182173,0.271803


In [69]:
px.line(c_df2, x='time', y='tri_hedge')